In [2]:
import subprocess
import osmium
import os
from geopy.geocoders import Nominatim

In [ ]:
city_full = 'Recife, Brazil'
city = 'rec'
filename = 'data/osm/%s.osm.pbf'%(city)
input_filename = 'data/osm/south-america-latest.osm.pbf'

geolocator = Nominatim(user_agent='uic')
location = geolocator.geocode(city_full).raw

bbox = [float(x) for x in location['boundingbox']]
bbox = [{'lat': bbox[0], 'lon': bbox[2]}, {'lat': bbox[1], 'lon': bbox[3]}] # bottom left, top right

In [ ]:
aux = '%f,%f,%f,%f'%(bbox[0]['lon'],bbox[0]['lat'],bbox[1]['lon'],bbox[1]['lat'])
proc = subprocess.call(['osmium', 'extract', '-b', aux, '-o', filename, '--overwrite', input_filename], shell=True) # WSL wasn't required!

### Historical data extract from: https://osm-internal.download.geofabrik.de

In [7]:
city_full = 'Recife, Brazil'
city = 'rec'
filename = 'data/osm/%s_historical.osm.pbf'%(city)
input_filename = 'data/osm/south-america-internal.osh.pbf'

geolocator = Nominatim(user_agent='uic')
location = geolocator.geocode(city_full).raw

bbox = [float(x) for x in location['boundingbox']]
bbox = [{'lat': bbox[0], 'lon': bbox[2]}, {'lat': bbox[1], 'lon': bbox[3]}] # bottom left, top right

In [9]:
aux = '%f,%f,%f,%f'%(bbox[0]['lon'],bbox[0]['lat'],bbox[1]['lon'],bbox[1]['lat'])
proc = subprocess.call(['osmium', 'extract', '-b', aux, '-H', '-o', filename, '--overwrite', input_filename], shell=True) # WSL wasn't required! Extra -H flag for historical data